In [1]:
import numpy as np

Some first names and surnames are given, based on the English (lowercase) alphabet.

In [2]:
fnames = "david", "anton", "fred", "jim", "barry"
snames = "barber", "ilsung", "fox", "chain", "fitzwilliam", "quinceadams", "grafvonunterhosen"

In [3]:
alphabet = "a", "b", "c", "d", "e", "f", "g", "h", "i", "j", "k", "l", "m", "n", "o", "p", "q", "r", "s", "t", "u", "v", "w", "x", "y", "z"

In [4]:
sequence = tuple((open("sequence.txt", "r")).read())
T = len(sequence)

In [23]:
tp = ""
for char in sequence:
    tp += "{},".format(alphabet.index(char) + 1)
tp

'11,10,18,19,5,11,25,4,8,22,9,25,21,12,3,6,11,17,6,11,25,7,10,2,3,5,7,16,1,13,6,15,9,23,23,2,7,13,9,4,21,18,19,10,7,17,21,26,4,3,4,14,5,16,6,9,24,26,17,3,26,14,4,5,21,9,18,21,6,10,3,14,22,13,8,4,13,25,1,6,26,23,4,17,1,6,22,13,3,26,8,18,22,1,11,7,12,20,5,23,5,9,26,15,15,18,11,15,5,18,24,16,23,24,21,12,23,13,14,18,22,26,26,15,21,5,12,10,13,13,9,19,13,3,5,2,1,13,11,24,19,21,23,23,20,8,26,4,13,18,20,16,17,5,9,26,3,14,1,4,3,13,19,21,23,23,20,10,18,19,13,10,19,7,26,4,8,6,11,12,10,26,9,12,12,26,17,25,5,17,16,17,22,6,7,6,20,22,26,13,1,11,5,2,23,18,11,21,4,20,16,12,20,26,23,9,12,6,11,2,11,7,3,2,4,10,5,3,26,18,2,6,16,23,23,8,1,12,1,11,20,9,9,21,12,22,15,2,1,19,2,16,6,14,17,6,13,15,4,1,14,15,22,9,4,8,23,15,20,9,3,25,4,23,13,9,14,10,2,16,11,23,6,14,20,26,6,26,14,6,19,16,12,25,24,1,7,23,18,12,14,7,19,10,16,6,1,7,12,6,19,25,10,22,13,20,14,3,15,2,25,1,12,10,6,20,8,5,3,17,4,16,12,19,11,17,10,3,19,13,11,19,14,13,26,21,26,11,12,12,10,20,15,1,9,11,26,13,25,8,9,5,16,14,5,15,20,21,15,20,16,20,5,15,24,7,1,1

In [5]:
states_to_char = {}
states_to_char["r-0"] = " "
states_to_char["r-1"] = " "
states = ["r-0"]
for i, name in enumerate(fnames):
    for j, char in enumerate(name):
        state = "f-{}-{}".format(i, j)
        states_to_char[state] = char
        states.append(state)
states += ["r-1"]
for i, name in enumerate(snames):
    for j, char in enumerate(name):
        state = "s-{}-{}".format(i, j)
        states_to_char[state] = char
        states.append(state)

The transitions operate as follows:
* A state of either random character generator switches to another state of the same random character generator with probability $0.8 \times \frac{1}{26}$.
* A state of either random character generator switches to the state of the first character of a name with probability $0.2 \times \frac{1}{|\mathcal{N}|}$, where $\mathcal{N}$ is the set of names (first names for the first random generator and surname for the second generator)
* A state of a character of a name switches to the state of the next character for that name with probability $1.0$, unless it's the last character in which case it would switch to the state of a character of the following random generator with probability $\frac{1}{26}$

In [6]:
A = {}
for a in states:
    A[a] = {}
    for b in states:
        if a == "r-0":
            if b == "r-0":
                A[a][b] = 0.8
            elif b.startswith("f-") and b.endswith("-0"):
                A[a][b] = 0.2 / len(fnames)
            else:
                A[a][b] = 0.0
        elif a == "r-1":
            if b == "r-1":
                A[a][b] = 0.8
            elif b.startswith("s-") and b.endswith("-0"):
                A[a][b] = 0.2 / len(snames)
            else:
                A[a][b] = 0.0
        elif a.startswith("f-"):
            name_i, char_i = int((a.split("-"))[1]), int((a.split("-"))[2])
            next_name_state = "f-{}-{}".format(str(name_i), str(char_i + 1))
            if next_name_state in states:
                A[a][b] = 1 if b == next_name_state else 0
            else:
                A[a][b] = 1 if b.startswith("r-1") else 0
        elif a.startswith("s-"):
            name_i, char_i = int((a.split("-"))[1]), int((a.split("-"))[2])
            next_name_state = "s-{}-{}".format(str(name_i), str(char_i + 1))
            if next_name_state in states:
                A[a][b] = 1 if b == next_name_state else 0
            else:
                A[a][b] = 1 if b.startswith("r-0") else 0

In [7]:
B = {}
for state in states:
    B[state] = {}
    for char in alphabet:
        if state.startswith("r-"):
            B[state][char] = 1 / 26
        else:
            B[state][char] = (char == states_to_char[state]) * 0.3 + (char != states_to_char[state]) * 0.7 / 25

In [17]:
for state in states:
    r = ""
    for char in alphabet:
        r += str(B[state][char]) + ","
    print(r[:-1])

0.038461538461538464,0.038461538461538464,0.038461538461538464,0.038461538461538464,0.038461538461538464,0.038461538461538464,0.038461538461538464,0.038461538461538464,0.038461538461538464,0.038461538461538464,0.038461538461538464,0.038461538461538464,0.038461538461538464,0.038461538461538464,0.038461538461538464,0.038461538461538464,0.038461538461538464,0.038461538461538464,0.038461538461538464,0.038461538461538464,0.038461538461538464,0.038461538461538464,0.038461538461538464,0.038461538461538464,0.038461538461538464,0.038461538461538464
0.027999999999999997,0.027999999999999997,0.027999999999999997,0.3,0.027999999999999997,0.027999999999999997,0.027999999999999997,0.027999999999999997,0.027999999999999997,0.027999999999999997,0.027999999999999997,0.027999999999999997,0.027999999999999997,0.027999999999999997,0.027999999999999997,0.027999999999999997,0.027999999999999997,0.027999999999999997,0.027999999999999997,0.027999999999999997,0.027999999999999997,0.027999999999999997,0.0279999

In [8]:
I = {}
for state in states:
    I[state] = 1 if state == "r-0" else 0

In [9]:
# from decimal import Decimal

# def viterbi():
#     t1, t2 = {}, {}
#     for i, state in enumerate(states):
#         t1[state], t2[state] = [], []
#         t1[state] += [Decimal(I[state] * B[state][sequence[0]])]
#         t2[state] += [Decimal(0)]
#     for i_, o in enumerate(sequence[1:]):
#         i = i_ + 1
#         for j, state in enumerate(states):
#             maxk, argmaxk = 0, 0
#             for k, s in enumerate(states):
#                 x = t1[s][i - 1] * Decimal(A[s][state]) * Decimal(B[state][sequence[i]])
#                 if x > maxk:
#                     maxk = x
#                     argmaxk = s
#             t1[state] += [maxk]
#             t2[state] += [argmaxk]
#     z_max, z_argmax = 0, 0
#     for i, state in enumerate(states):
#         if t1[state][T - 1] > z_max:
#             z_max = t1[state][T - 1]
#             z_argmax = state
#     z = [z_argmax]
#     x = [states_to_char[z_argmax]]
#     for t in range(T - 1, 0, -1):
#         v = t2[z[0]][t]
#         z.insert(0, v)
#         x.insert(0, states_to_char[v])
#     return x

# v = viterbi()

In [32]:
cs = "1,1,1,1,1,1,1,2,3,4,5,6,24,24,24,24,24,24,24,24,56,57,58,59,60,61,62,63,64,65,66,1,1,1,1,2,3,4,5,6,24,24,24,24,24,56,57,58,59,60,61,62,63,64,65,66,1,1,1,1,1,1,2,3,4,5,6,24,24,24,24,24,24,24,24,24,24,45,46,47,48,49,50,51,52,53,54,55,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,16,17,18,24,45,46,47,48,49,50,51,52,53,54,55,1,1,1,1,1,19,20,21,22,23,24,24,24,24,24,24,24,24,24,24,24,24,56,57,58,59,60,61,62,63,64,65,66,1,1,1,1,16,17,18,24,24,24,24,24,24,24,45,46,47,48,49,50,51,52,53,54,55,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,19,20,21,22,23,24,24,45,46,47,48,49,50,51,52,53,54,55,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,7,8,9,10,11,24,24,24,24,25,26,27,28,29,30,1,1,1,1,1,1,1,2,3,4,5,6,24,24,24,24,24,24,24,40,41,42,43,44,1,1,1,1,1,7,8,9,10,11,24,24,24,24,24,24,24,24,24,31,32,33,34,35,36,1,1,1,1,1,1,1,1,1,1,16,17,18,24,24,24,24,24,24,24,24,24,24,24,24,24,24,24,24,31,32,33,34,35,36,1,1,1,1,1,1,1,1,1,1,1,1,7,8,9,10,11,24,24,24,24,24,24,67,68,69,70,71,72,73,74,75,76,77,78,79,80,81,82,83,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,19,20,21,22,23,24,24,24,24,24,24,24,24,24,24,24,24,24,24,31,32,33,34,35,36,1,1,1,1,1,1,19,20,21,22,23,24,24,24,67,68,69,70,71,72,73,74,75,76,77,78,79,80,81,82,83,1,16,17,18,24,24,24,24,45,46,47,48,49,50,51,52,53,54,55,1,1,1,1,1,2,3,4,5,6,24,24,40,41,42,43,44,1,1,1,1,1,7,8,9,10,11,24,24,56,57,58,59,60,61,62,63,64,65,66,1,1,1,1,19,20,21,22,23,24,24,24,45,46,47,48,49,50,51,52,53,54,55,1,1,1,1,1,12,13,14,15,24,24,24,24,24,67,68,69,70,71,72,73,74,75,76,77,78,79,80,81,82,83,1,1,1,1,1,1,1,1,1,12,13,14,15,24,24,24,40,41,42,43,44,1,1,1,12,13,14,15,24,67,68,69,70,71,72,73,74,75,76,77,78,79,80,81,82,83,1,1,1,1,1,1,1,1,12,13,14,15,24,67,68,69,70,71,72,73,74,75,76,77,78,79,80,81,82,83,1,1,1,1,1,1,1,1,7,8,9,10,11,24,24,24,24,24,24,24,24,24,24,24,24,24,24,24,56,57,58,59,60,61,62,63,64,65,66,1,1,1,1,1,12,13,14,15,24,24,25,26,27,28,29,30,1,1,1,1,7,8,9,10,11,24,24,24,24,24,37,38,39,1,1,19,20,21,22,23,24,24,24,24,24,24,40,41,42,43,44,1,1,1,1,1,1,1,1,1,1,1,16,17,18,24,24,24,24,24,67,68,69,70,71,72,73,74,75,76,77,78,79,80,81,82,83,1,16,17,18,24,24,45,46,47,48,49,50,51,52,53,54,55,1,1,19,20,21,22,23,24,24,24,24,24,24,45,46,47,48,49,50,51,52,53,54,55,1,1,1,1,1,1,19,20,21,22,23,24,24,24,24,24,24,24,24,31,32,33,34,35,36,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,19,20,21,22,23,24,24,24,67,68,69,70,71,72,73,74,75,76,77,78,79,80,81,82,83,1,1,1,1,1,1,1,12,13,14,15,24,24,67,68,69,70,71,72,73,74,75,76,77,78,79,80,81,82,83,1,1,1,1,1,12,13,14,15,24,24,24,24,24,24,67,68,69,70,71,72,73,74,75,76,77,78,79,80,81,82,83,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,7,8,9,10,11,24,24,24,24,24,40,41,42,43,44,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,16,17,18,24,24,24,24,24,24,24,37,38,39,1,1,1,1,1,16,17,18,24,24,24,24,24,24,24,24,24,24,24,31,32,33,34,35,36,1,1,1,1,1,1,16,17,18,24,25,26,27,28,29,30,1,1,12,13,14,15,24,24,24,24,24,67,68,69,70,71,72,73,74,75,76,77,78,79,80,81,82,83,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,12,13,14,15,24,24,24,56,57,58,59,60,61,62,63,64,65,66,1,1,1,12,13,14,15,24,24,24,24,24,24,24,56,57,58,59,60,61,62,63,64,65,66,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,16,17,18,24,67,68,69,70,71,72,73,74,75,76,77,78,79,80,81,82,83,1,1,1,1,16,17,18,24,45,46,47,48,49,50,51,52,53,54,55,1,1,1,1,1,1,1,1,19,20,21,22,23,24,24,24,24,24,24,24,24,24,24,24,24,24,24,24,24,24,24,24,24,40,41,42,43,44,1,1,1,1,1,16,17,18,24,67,68,69,70,71,72,73,74,75,76,77,78,79,80,81,82,83,1,1,1,1,1,1,1,1,1,19,20,21,22,23,24,24,24,24,24,24,24,24,24,24,24,24,24,24,24,24,24,24,24,24,24,24,67,68,69,70,71,72,73,74,75,76,77,78,79,80,81,82,83,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,7,8,9,10,11,24,45,46,47,48,49,50,51,52,53,54,55,1,1,1,1,1,1,1,1,1,2,3,4,5,6,24,24,56,57,58,59,60,61,62,63,64,65,66,1,1,1,1,1,1,1,1,2,3,4,5,6,24,24,24,24,24,45,46,47,48,49,50,51,52,53,54,55,1,1,1,1,1,1,16,17,18,24,24,24,40,41,42,43,44,1,1,16,17,18,24,24,37,38,39,1,1,1,1,1,1,1,1,7,8,9,10,11,24,45,46,47,48,49,50,51,52,53,54,55,1,12,13,14,15,24,24,24,24,45,46,47,48,49,50,51,52,53,54,55,1,1,1,12,13,14,15,24,24,24,24,24,25,26,27,28,29,30,1,1,1,1,19,20,21,22,23,24,37,38,39,1,1,19,20,21,22,23,24,24,24,24,24,24,67,68,69,70,71,72,73,74,75,76,77,78,79,80,81,82,83,1,1,1,1,1,12,13,14,15,24,37,38,39,1,1,7,8,9,10,11,24,24,24,45,46,47,48,49,50,51,52,53,54,55,1,1,1,1,2,3,4,5,6,24,24,24,24,24,24,24,24,24,24,24,24,24,24,24,24,24,24,24,40,41,42,43,44,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,19,20,21,22,23,24,24,24,67,68,69,70,71,72,73,74,75,76,77,78,79,80,81,82,83,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,12,13,14,15,24,40,41,42,43,44,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,19,20,21,22,23,24,24,24,24,24,24,24,24,24,24,24,24,24,24,24,24,25,26,27,28,29,30,1,1,1,1,1,1,1,1,1,2,3,4,5,6,24,24,24,24,24,24,24,24,24,24,24,24,24,24,67,68,69,70,71,72,73,74,75,76,77,78,79,80,81,82,83,1,1,1,1,1,1,1,2,3,4,5,6,24,24,24,24,24,24,24,24,24,24,24,24,24,24,24,24,24,24,24,24,24,24,24,24,24,45,46,47,48,49,50,51,52,53,54,55,1,1,1,1,1,12,13,14,15,24,24,24,24,24,24,24,24,25,26,27,28,29,30,1,1,1,1,1,1,2,3,4,5,6,24,24,24,24,24,24,24,24,24,24,24,24,24,45,46,47,48,49,50,51,52,53,54,55,1,1,1,1,1,1,1,1,1,1,1,1,1,16,17,18,24,24,24,25,26,27,28,29,30,1,1,1,1,19,20,21,22,23,24,24,24,24,24,40,41,42,43,44,1,1,1,1,2,3,4,5,6,24,37,38,39,1,1,1,1,7,8,9,10,11,24,24,24,24,25,26,27,28,29,30,1,1,1,2,3,4,5,6,24,24,24,24,24,24,24,40,41,42,43,44,1,1,1,7,8,9,10,11,24,24,24,24,24,24,24,24,45,46,47,48,49,50,51,52,53,54,55,1,1,1,1,1,12,13,14,15,24,24,24,24,24,24,24,24,24,24,24,24,24,24,40,41,42,43,44,1,1,1,16,17,18,24,24,45,46,47,48,49,50,51,52,53,54,55,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,16,17,18,24,24,24,24,24,24,24,24,24,56,57,58,59,60,61,62,63,64,65,66,1,1,2,3,4,5,6,24,24,67,68,69,70,71,72,73,74,75,76,77,78,79,80,81,82,83,1,1,1,1,1,1,1,1,1,16,17,18,24,45,46,47,48,49,50,51,52,53,54,55,1,1,7,8,9,10,11,24,24,24,24,24,24,24,24,24,25,26,27,28,29,30,1,1,1,1,1,1,1,16,17,18,24,24,24,24,24,24,24,24,24,24,24,24,24,24,24,24,24,24,24,24,24,37,38,39,1,1,7,8,9,10,11,24,24,24,24,24,24,24,24,37,38,39,1,1,1,1,1,16,17,18,24,24,24,24,24,24,40,41,42,43,44,1,1,1,1,19,20,21,22,23,24,24,24,24,24,24,24,24,24,40,41,42,43,44,1,1,1,1,1,1,1,1,1,1,1,1,1,1,7,8,9,10,11,24,45,46,47,48,49,50,51,52,53,54,55,1,1,1,1,1,1,1,12,13,14,15,24,24,24,24,40,41,42,43,44,1,12,13,14,15,24,24,24,24,24,24,24,24,45,46,47,48,49,50,51,52,53,54,55,1,1,2,3,4,5,6,24,24,24,45,46,47,48,49,50,51,52,53,54,55,1,1,1,1,2,3,4,5,6,24,37,38,39,1,1,1,12,13,14,15,24,31,32,33,34,35,36,1,1,1,1,1,1,16,17,18,24,24,24,24,67,68,69,70,71,72,73,74,75,76,77,78,79,80,81,82,83,1,1,1,7,8,9,10,11,24,24,24,24,24,24,24,24,24,24,24,24,24,24,24,24,31,32,33,34,35,36,1,1,1,1,1,7,8,9,10,11,24,24,40,41,42,43,44,1,1,1,1,1,1,16,17,18,24,24,56,57,58,59,60,61,62,63,64,65,66,1,19,20,21,22,23,24,24,24,24,37,38,39,1,1,16,17,18,24,24,24,24,24,24,24,24,24,24,24,24,24,24,24,67,68,69,70,71,72,73,74,75,76,77,78,79,80,81,82,83,1,1,1,1,1,19,20,21,22,23,24,24,24,24,67,68,69,70,71,72,73,74,75,76,77,78,79,80,81,82,83,1,19,20,21,22,23,24,24,24,24,24,24,24,24,24,24,24,24,24,25,26,27,28,29,30,1,1,1,1,1,1,2,3,4,5,6,24,24,24,24,24,24,24,24,24,56,57,58,59,60,61,62,63,64,65,66,1,1,1,1,1,19,20,21,22,23,24,37,38,39,1,1,1,7,8,9,10,11,24,24,24,31,32,33,34,35,36,1,1,1,1,19,20,21,22,23,24,24,25,26,27,28,29,30,1,1,12,13,14,15,24,24,24,67,68,69,70,71,72,73,74,75,76,77,78,79,80,81,82,83,1,1,7,8,9,10,11,24,40,41,42,43,44,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,16,17,18,24,24,24,24,24,24,24,24,31,32,33,34,35,36,1,1,16,17,18,24,24,24,24,24,67,68,69,70,71,72,73,74,75,76,77,78,79,80,81,82,83,1,1,1,1,1,1,1,1,1,12,13,14,15,24,24,24,24,67,68,69,70,71,72,73,74,75,76,77,78,79,80,81,82,83,1,1,1,1,1,12,13,14,15,24,24,24,40,41,42,43,44,1,1,2,3,4,5,6,24,24,24,24,24,24,24,24,24,67,68,69,70,71,72,73,74,75,76,77,78,79,80,81,82,83,1,1,1,19,20,21,22,23,24,24,24,24,24,24,24,24,24,37,38,39,1,1,1,1,1,1,1,1,1,1,1,1,1,1,19,20,21,22,23,24,24,24,24,24,24,24,24,24,24,24,24,24,24,24,24,24,24,24,24,24,24,24,24,24,56,57,58,59,60,61,62,63,64,65,66,1,1,1,1,1,1,19,20,21,22,23,24,24,24,24,24,24,24,24,24,24,24,40,41,42,43,44,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,2,3,4,5,6,24,24,24,24,24,24,24,24,24,24,56,57,58,59,60,61,62,63,64,65,66,1,1,19,20,21,22,23,24,24,56,57,58,59,60,61,62,63,64,65,66,1,1,1,1,1,16,17,18,24,24,24,24,24,24,24,24,24,24,24,24,24,40,41,42,43,44,1,16,17,18,24,24,24,24,24,24,24,24,24,24,24,24,24,24,24,24,45,46,47,48,49,50,51,52,53,54,55,1,1,1,1,1,7,8,9,10,11,24,24,24,24,24,24,24,24,24,24,24,24,24,45,46,47,48,49,50,51,52,53,54,55,1,1,1,1,1,1,1,1,2,3,4,5,6,24,24,24,67,68,69,70,71,72,73,74,75,76,77,78,79,80,81,82,83,1,1,1,1,1,1,1,16,17,18,24,40,41,42,43,44,1,1,16,17,18,24,24,24,24,24,24,24,40,41,42,43,44,1,1,1,1,1,2,3,4,5,6,24,37,38,39,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,12,13,14,15,24,24,24,24,24,24,24,40,41,42,43,44,1,1,1,1,1,1,1,1,1,1,1,1,19,20,21,22,23,24,24,24,24,24,24,24,24,24,24,24,24,24,24,24,24,24,24,24,24,24,24,56,57,58,59,60,61,62,63,64,65,66,1,1,1,1,1,1,1,1,1,1,1,1,1,1,7,8,9,10,11,24,24,24,56,57,58,59,60,61,62,63,64,65,66,1,7,8,9,10,11,24,24,24,56,57,58,59,60,61,62,63,64,65,66,1,1,1,1,1,1,1,1,1,1,1,1,19,20,21,22,23,24,67,68,69,70,71,72,73,74,75,76,77,78,79,80,81,82,83,1,1,12,13,14,15,24,24,24,24,24,24,24,24,24,24,24,24,24,24,24,24,24,24,24,24,24,24,24,24,37,38,39,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,16,17,18,24,24,24,24,56,57,58,59,60,61,62,63,64,65,66,1,1,16,17,18,24,24,40,41,42,43,44,1,1,1,1,1,1,1,1,1,7,8,9,10,11,24,24,24,24,24,24,24,24,24,25,26,27,28,29,30,1,1,16,17,18,24,40,41,42,43,44,1,1,1,1,1,1,1,1,1,1,2,3,4,5,6,24,67,68,69,70,71,72,73,74,75,76,77,78,79,80,81,82,83,1,1,1,1,1,1,1,1,1,1,1,1,16,17,18,24,24,24,25,26,27,28,29,30,1,1,1,1,1,1,1,1,1,2,3,4,5,6,24,24,24,24,24,24,67,68,69,70,71,72,73,74,75,76,77,78,79,80,81,82,83,1,1,1,12,13,14,15,24,24,24,24,24,56,57,58,59,60,61,62,63,64,65,66,1,1,1,1,1,1,1,1,1,1,1,12,13,14,15,24,24,24,24,24,24,24,24,24,24,24,24,31,32,33,34,35,36,1,1,1,2,3,4,5,6,24,24,24,24,24,24,45,46,47,48,49,50,51,52,53,54,55,1,1,1,1,1,1,19,20,21,22,23,24,56,57,58,59,60,61,62,63,64,65,66,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,2,3,4,5,6,24,24,45,46,47,48,49,50,51,52,53,54,55,1,1,1,1,1,1,2,3,4,5,6,24,24,24,24,24,24,40,41,42,43,44,1,1,1,1,1,1,1,1,1,1,1,1,2,3,4,5,6,24,24,31,32,33,34,35,36,1,1,1,1,1,1,1,19,20,21,22,23,24,24,24,24,24,24,24,24,24,24,24,24,24,24,24,45,46,47,48,49,50,51,52,53,54,55,1,1,1,1,1,1,1,1,1,1,1,12,13,14,15,24,24,24,24,24,24,45,46,47,48,49,50,51,52,53,54,55,1,1,1,1,1,1,1,1,1,1,16,17,18,24,67,68,69,70,71,72,73,74,75,76,77,78,79,80,81,82,83,1,1,1,1,1,1,1,16,17,18,24,24,24,24,24,24,24,24,24,56,57,58,59,60,61,62,63,64,65,66,1,1,2,3,4,5,6,24,24,24,24,24,24,24,24,24,24,24,24,24,45,46,47,48,49,50,51,52,53,54,55,1,1,1,1,16,17,18,24,24,24,24,24,24,24,24,24,24,24,24,24,67,68,69,70,71,72,73,74,75,76,77,78,79,80,81,82,83,1,1,1,1,1,1,1,1,2,3,4,5,6,24,24,25,26,27,28,29,30,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,16,17,18,24,24,24,24,24,24,24,40,41,42,43,44,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,19,20,21,22,23,24,24,24,24,24,24,24,24,24,24,24,24,24,24,24,24,24,24,24,24,25,26,27,28,29,30,1,1,1,1,7,8,9,10,11,24,24,24,24,24,24,24,45,46,47,48,49,50,51,52,53,54,55,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,12,13,14,15,24,24,24,24,56,57,58,59,60,61,62,63,64,65,66,1,1,1,16,17,18,24,24,24,24,24,24,56,57,58,59,60,61,62,63,64,65,66,1,1,1,1,1,16,17,18,24,24,24,24,24,24,24,37,38,39,1,1,2,3,4,5,6,24,24,24,24,24,40,41,42,43,44,1,1,1,1,1,1,1,1,1,1,1,12,13,14,15,24,25,26,27,28,29,30,1,1,1,1,2,3,4,5,6,24,24,24,31,32,33,34,35,36,1,1,1,1,12,13,14,15,24,24,24,24,24,24,24,24,40,41,42,43,44,1,7,8,9,10,11,24,24,24,24,24,24,24,24,24,24,24,45,46,47,48,49,50,51,52,53,54,55,1,1,1,1,1,1,1,16,17,18,24,24,24,24,24,24,24,25,26,27,28,29,30,1,1,1,1,1,1,1,1,1,1,1,1,1,19,20,21,22,23,24,24,24,24,24,24,24,56,57,58,59,60,61,62,63,64,65,66,1,1,1,1,1,1,1,1,1,1,1,1,1,1,16,17,18,24,24,24,24,24,24,24,24,24,24,24,67,68,69,70,71,72,73,74,75,76,77,78,79,80,81,82,83,1,1,16,17,18,24,24,40,41,42,43,44,1,1,1,1,16,17,18,24,24,67,68,69,70,71,72,73,74,75,76,77,78,79,80,81,82,83,1,1,12,13,14,15,24,24,24,24,67,68,69,70,71,72,73,74,75,76,77,78,79,80,81,82,83,1,1,1,1,1,1,12,13,14,15,24,67,68,69,70,71,72,73,74,75,76,77,78,79,80,81,82,83,1,1,1,7,8,9,10,11,24,45,46,47,48,49,50,51,52,53,54,55,1,19,20,21,22,23,24,24,24,24,24,24,24,24,67,68,69,70,71,72,73,74,75,76,77,78,79,80,81,82,83,1,1,1,16,17,18,24,56,57,58,59,60,61,62,63,64,65,66,1,1,19,20,21,22,23,24,24,24,24,24,24,24,37,38,39,1,1,1,1,19,20,21,22,23,24,24,45,46,47,48,49,50,51,52,53,54,55,1,1,16,17,18,24,24,24,24,24,24,24,24,45,46,47,48,49,50,51,52,53,54,55,1,1,1,19,20,21,22,23,24,24,24,24,37,38,39,1,1,1,1,12,13,14,15,24,37,38,39,1,1,1,1,1,1,1,1,12,13,14,15,24,24,24,24,24,24,24,24,24,24,24,24,37,38,39,1,1,1,1,1,1,19,20,21,22,23,24,24,24,24,24,24,24,24,24,24,24,24,24,24,24,24,67,68,69,70,71,72,73,74,75,76,77,78,79,80,81,82,83,1,1,1,1,1,1,1,1,1,1,1,1,1,19,20,21,22,23,24,24,24,24,45,46,47,48,49,50,51,52,53,54,55,1,19,20,21,22,23,24,24,24,24,24,24,24,24,24,24,24,24,24,25,26,27,28,29,30,1,1,16,17,18,24,24,24,24,24,24,24,24,31,32,33,34,35,36,1,1,1,1,7,8,9,10,11,24,24,24,40,41,42,43,44,1,1,1,1,1,1,1,12,13,14,15,24,24,24,24,24,24,24,24,24,24,24,24,24,24,24,56,57,58,59,60,61,62,63,64,65,66,1,1,1,12,13,14,15,24,24,24,24,24,45,46,47,48,49,50,51,52,53,54,55,1,1,1,1,1,16,17,18,24,24,24,24,24,37,38,39,1,1,12,13,14,15,24,24,24,24,24,37,38,39,1,1,1,1,1,2,3,4,5,6,24,24,24,37,38,39,1,1,1,16,17,18,24,24,24,24,24,24,24,24,24,40,41,42,43,44,1,2,3,4,5,6,24,24,24,24,24,24,24,40,41,42,43,44,1,1,1,1,1,7,8,9,10,11,24,24,24,24,24,24,24,24,24,24,24,24,24,45,46,47,48,49,50,51,52,53,54,55,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,7,8,9,10,11,24,24,24,24,24,24,24,24,24,24,24,24,24,24,24,25,26,27,28,29,30,1,1,1,7,8,9,10,11,24,24,24,24,24,56,57,58,59,60,61,62,63,64,65,66,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,19,20,21,22,23,24,24,24,24,24,31,32,33,34,35,36,1,1,1,7,8,9,10,11,24,40,41,42,43,44,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,7,8,9,10,11,24,24,31,32,33,34,35,36,1,1,16,17,18,24,24,24,24,24,24,24,24,24,24,24,45,46,47,48,49,50,51,52,53,54,55,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,16,17,18,24,24,24,24,24,24,24,24,24,40,41,42,43,44,1,1,1,1,1,1,1,1,1,1,1,1,7,8,9,10,11,24,24,24,24,24,24,24,24,24,24,24,24,24,24,24,24,24,31,32,33,34,35,36,1,1,19,20,21,22,23,24,24,24,67,68,69,70,71,72,73,74,75,76,77,78,79,80,81,82,83,1,1,1,1,12,13,14,15,24,31,32,33,34,35,36,1,1,1,16,17,18,24,45,46,47,48,49,50,51,52,53,54,55,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,7,8,9,10,11,24,24,25,26,27,28,29,30,1,7,8,9,10,11,24,24,24,45,46,47,48,49,50,51,52,53,54,55,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,16,17,18,24,37,38,39,1,1,1,1,1,1,1,19,20,21,22,23,24,25,26,27,28,29,30,1,1,19,20,21,22,23,24,24,67,68,69,70,71,72,73,74,75,76,77,78,79,80,81,82,83,1,1,1,1,1,1,1,1,1,2,3,4,5,6,24,24,67,68,69,70,71,72,73,74,75,76,77,78,79,80,81,82,83,1,1,1,1,1,1,1,1,2,3,4,5,6,24,56,57,58,59,60,61,62,63,64,65,66,1,1,1,7,8,9,10,11,24,24,24,24,24,24,24,56,57,58,59,60,61,62,63,64,65,66,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,12,13,14,15,24,67,68,69,70,71,72,73,74,75,76,77,78,79,80,81,82,83,1,19,20,21,22,23,24,24,24,24,24,24,24,24,24,31,32,33,34,35,36,1,2,3,4,5,6,24,24,24,24,24,24,24,24,24,24,24,24,24,45,46,47,48,49,50,51,52,53,54,55,1,1,1,1,1,1,19,20,21,22,23,24,24,45,46,47,48,49,50,51,52,53,54,55,1,1,1,1,1,1,1,1,1,1,1,1,12,13,14,15,24,24,24,24,24,24,24,24,24,24,24,24,24,24,24,24,24,24,24,24,24,24,24,45,46,47,48,49,50,51,52,53,54,55,1,1,1,1,16,17,18,24,31,32,33,34,35,36,1,1,1,1,1,1,2,3,4,5,6,24,25,26,27,28,29,30,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,7,8,9,10,11,24,25,26,27,28,29,30,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,16,17,18,24,24,24,56,57,58,59,60,61,62,63,64,65,66,1,1,1,1,1,1,1,7,8,9,10,11,24,24,24,24,67,68,69,70,71,72,73,74,75,76,77,78,79,80,81,82,83,1,7,8,9,10,11,24,45,46,47,48,49,50,51,52,53,54,55,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,7,8,9,10,11,24,24,24,24,24,24,24,24,24,24,24,24,24,24,24,24,24,40,41,42,43,44,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,2,3,4,5,6,24,24,24,24,24,24,24,24,24,24,45,46,47,48,49,50,51,52,53,54,55,1,1,16,17,18,24,24,24,45,46,47,48,49,50,51,52,53,54,55,1,1,1,1,1,1,1,1,1,1,1,1,1,7,8,9,10,11,24,24,24,24,24,24,24,24,24,24,24,24,24,24,24,24,24,24,24,24,24,25,26,27,28,29,30,1,1,1,1,1,1,1,1,7,8,9,10,11,24,24,24,24,24,24,56,57,58,59,60,61,62,63,64,65,66,1,1,1,1,1,12,13,14,15,24,24,24,24,24,24,24,24,24,31,32,33,34,35,36,1,1,1,1,1,1,1,1,2,3,4,5,6,24,24,24,45,46,47,48,49,50,51,52,53,54,55,1,1,1,1,1,12,13,14,15,24,24,24,24,24,45,46,47,48,49,50,51,52,53,54,55,1,1,16,17,18,24,24,24,24,24,24,24,24,24,24,24,24,24,24,24,24,24,24,24,24,24,24,24,56,57,58,59,60,61,62,63,64,65,66,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,16,17,18,24,24,24,24,45,46,47,48,49,50,51,52,53,54,55,1,7,8,9,10,11,24,24,24,24,24,24,24,24,24,24,24,24,24,24,24,24,24,24,24,24,24,24,24,24,24,67,68,69,70,71,72,73,74,75,76,77,78,79,80,81,82,83,1,1,1,16,17,18,24,24,24,24,67,68,69,70,71,72,73,74,75,76,77,78,79,80,81,82,83,1,1,1,1,1,1,16,17,18,24,24,24,24,24,24,24,45,46,47,48,49,50,51,52,53,54,55,1,1,1,1,1,1,1,1,1,1,1,1,1,16,17,18,24,24,24,24,24,24,24,24,45,46,47,48,49,50,51,52,53,54,55,1,1,1,1,1,1,1,1,1,1,1,1,7,8,9,10,11,24,24,24,24,24,24,24,24,24,24,24,24,24,24,40,41,42,43,44,1,7,8,9,10,11,24,24,24,67,68,69,70,71,72,73,74,75,76,77,78,79,80,81,82,83,1,1,1,1,1,1,1,1,1,1,1,1,16,17,18,24,31,32,33,34,35,36,1,1,12,13,14,15,24,24,24,24,24,24,24,24,24,24,24,24,24,24,24,24,24,24,67,68,69,70,71,72,73,74,75,76,77,78,79,80,81,82,83,1,16,17,18,24,24,24,24,24,24,24,24,24,24,24,24,24,24,24,24,24,24,24,24,24,24,24,67,68,69,70,71,72,73,74,75,76,77,78,79,80,81,82,83,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,7,8,9,10,11,24,24,45,46,47,48,49,50,51,52,53,54,55,1,1,2,3,4,5,6,24,24,24,24,24,24,24,24,24,24,67,68,69,70,71,72,73,74,75,76,77,78,79,80,81,82,83,1,1,1,2,3,4,5,6,24,24,24,24,24,24,24,24,37,38,39,1,1,16,17,18,24,24,24,24,24,24,24,24,24,24,24,24,24,24,24,24,40,41,42,43,44,1,1,2,3,4,5,6,24,56,57,58,59,60,61,62,63,64,65,66,1,1,1,1,1,1,1,1,1,1,1,1,1,2,3,4,5,6,24,24,24,24,24,24,24,24,24,24,24,24,24,24,24,24,67,68,69,70,71,72,73,74,75,76,77,78,79,80,81,82,83,1,1,1,1,1,2,3,4,5,6,24,24,37,38,39,1,1,1,1,1,1,1,7,8,9,10,11,24,24,24,24,24,37,38,39,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,16,17,18,24,24,37,38,39,1,1,1,1,1,1,1,1,1,1,1,1,1,12,13,14,15,24,67,68,69,70,71,72,73,74,75,76,77,78,79,80,81,82,83,1,1,1,1,19,20,21,22,23,24,24,24,24,67,68,69,70,71,72,73,74,75,76,77,78,79,80,81,82,83,1,1,1,1,1,1,1,1,1,7,8,9,10,11,24,24,24,67,68,69,70,71,72,73,74,75,76,77,78,79,80,81,82,83,1,1,1,1,1,1,1,7,8,9,10,11,24,67,68,69,70,71,72,73,74,75,76,77,78,79,80,81,82,83,1,1,1,1,1,12,13,14,15,24,24,24,24,24,24,40,41,42,43,44,1,1,1,1,1,1,1,1,1,1,1,1,1,2,3,4,5,6,24,24,24,25,26,27,28,29,30,1,1,1,1,1,1,1,1,1,19,20,21,22,23,24,40,41,42,43,44,1,1,1,2,3,4,5,6,24,24,24,24,25,26,27,28,29,30,1,1,1,1,1,1,1,1,1,1,1,1,1,7,8,9,10,11,24,24,24,24,24,24,24,24,24,24,24,24,24,24,24,24,56,57,58,59,60,61,62,63,64,65,66,1,1,1,16,17,18,24,56,57,58,59,60,61,62,63,64,65,66,1,1,1,1,1,1,1,1,7,8,9,10,11,24,31,32,33,34,35,36,1,1,19,20,21,22,23,24,24,24,24,24,24,24,24,24,40,41,42,43,44,1,1,1,1,16,17,18,24,24,24,24,24,24,24,24,56,57,58,59,60,61,62,63,64,65,66,1,1,1,1,7,8,9,10,11,24,24,24,24,24,24,24,24,24,24,24,24,24,24,24,45,46,47,48,49,50,51,52,53,54,55,1,1,1,1,1,1,1,1,16,17,18,24,24,24,24,24,24,24,24,24,24,24,24,24,24,24,24,24,24,24,67,68,69,70,71,72,73,74,75,76,77,78,79,80,81,82,83,1,1,1,1,2,3,4,5,6,24,45,46,47,48,49,50,51,52,53,54,55,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,2,3,4,5,6,24,31,32,33,34,35,36,1,16,17,18,24,24,37,38,39,1,1,1,1,1,1,7,8,9,10,11,24,67,68,69,70,71,72,73,74,75,76,77,78,79,80,81,82,83,1,1,1,1,1,1,1,19,20,21,22,23,24,24,24,24,24,24,24,40,41,42,43,44,1,1,1,16,17,18,24,24,24,24,24,24,40,41,42,43,44,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,2,3,4,5,6,24,24,24,24,24,24,24,24,24,24,24,24,24,40,41,42,43,44,1,1,1,1,1,1,12,13,14,15,24,24,24,24,24,37,38,39,1,16,17,18,24,24,67,68,69,70,71,72,73,74,75,76,77,78,79,80,81,82,83,1,1,1,1,1,2,3,4,5,6,24,24,24,24,24,24,24,24,56,57,58,59,60,61,62,63,64,65,66,1,1,7,8,9,10,11,24,24,24,24,56,57,58,59,60,61,62,63,64,65,66,1,1,1,1,1,1,1,1,1,2,3,4,5,6,24,24,40,41,42,43,44,1,1,1,1,1,1,1,16,17,18,24,24,24,24,24,24,24,25,26,27,28,29,30,1,1,1,1,1,1,19,20,21,22,23,24,24,24,24,24,24,24,24,24,24,24,24,24,24,24,24,24,24,24,24,24,24,31,32,33,34,35,36,1,1,7,8,9,10,11,24,67,68,69,70,71,72,73,74,75,76,77,78,79,80,81,82,83,1,1,1,1,16,17,18,24,24,24,67,68,69,70,71,72,73,74,75,76,77,78,79,80,81,82,83,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,12,13,14,15,24,24,24,24,24,24,24,45,46,47,48,49,50,51,52,53,54,55,1,1,19,20,21,22,23,24,24,24,24,24,37,38,39,1,1,16,17,18,24,24,24,24,31,32,33,34,35,36,1,1,1,1,1,1,1,16,17,18,24,24,45,46,47,48,49,50,51,52,53,54,55,1,1,1,1,1,1,1,1,1,1,1,1,19,20,21,22,23,24,24,24,24,24,24,24,24,24,24,67,68,69,70,71,72,73,74,75,76,77,78,79,80,81,82,83,1,1,1,1,1,1,1,1,1,1,16,17,18,24,24,24,24,24,24,24,24,24,24,24,24,24,24,24,24,24,24,24,24,24,24,24,45,46,47,48,49,50,51,52,53,54,55,1,1,1,1,1,1,1,1,16,17,18,24,24,31,32,33,34,35,36,1,1,1,1,19,20,21,22,23,24,24,24,24,24,24,24,24,24,37,38,39,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,2,3,4,5,6,24,24,24,24,24,40,41,42,43,44,1,1,1,1,1,1,1,1,1,1,1,1,1,19,20,21,22,23,24,24,24,24,24,40,41,42,43,44,1,1,1,1,1,1,1,1,1,1,1,1,1,1,19,20,21,22,23,24,24,24,24,24,24,24,24,24,24,24,67,68,69,70,71,72,73,74,75,76,77,78,79,80,81,82,83,1,1,1,1,1,1,1,1,1,1,1,1,12,13,14,15,24,24,24,24,24,24,24,45,46,47,48,49,50,51,52,53,54,55,1,1,2,3,4,5,6,24,24,24,45,46,47,48,49,50,51,52,53,54,55,1,1,1,1,1,1,16,17,18,24,24,24,24,24,67,68,69,70,71,72,73,74,75,76,77,78,79,80,81,82,83,1,1,19,20,21,22,23,24,24,56,57,58,59,60,61,62,63,64,65,66,1,1,1,1,1,1,1,1,1,1,1,1,12,13,14,15,24,67,68,69,70,71,72,73,74,75,76,77,78,79,80,81,82,83,1,1,7,8,9,10,11,24,24,56,57,58,59,60,61,62,63,64,65,66,1,2,3,4,5,6,24,24,37,38,39,1,1,1,1,12,13,14,15,24,37,38,39,1,1,12,13,14,15,24,24,24,24,24,24,45,46,47,48,49,50,51,52,53,54,55,1,1,19,20,21,22,23,24,24,45,46,47,48,49,50,51,52,53,54,55,1,1,1,1,1,1,1,1,2,3,4,5,6,24,24,24,24,24,24,24,24,24,24,56,57,58,59,60,61,62,63,64,65,66,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,12,13,14,15,24,24,24,24,24,24,24,24,24,24,24,24,24,24,24,24,24,24,24,24,24,67,68,69,70,71,72,73,74,75,76,77,78,79,80,81,82,83,1,1,1,1,16,17,18,24,24,24,24,24,24,24,24,40,41,42,43,44,1,1,1,1,16,17,18,24,45,46,47,48,49,50,51,52,53,54,55,1,1,1,1,1,1,1,1,1,1,7,8,9,10,11,24,24,24,24,67,68,69,70,71,72,73,74,75,76,77,78,79,80,81,82,83,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,2,3,4,5,6,24,24,24,24,24,24,24,25,26,27,28,29,30,1,1,1,1,1,1,1,1,1,1,1,12,13,14,15,24,24,24,24,24,24,24,24,24,24,24,24,24,45,46,47,48,49,50,51,52,53,54,55,1,1,1,1,1,1,1,1,1,1,1,1,1,16,17,18,24,24,24,24,24,67,68,69,70,71,72,73,74,75,76,77,78,79,80,81,82,83,1,1,1,1,1,1,1,1,16,17,18,24,24,24,40,41,42,43,44,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,19,20,21,22,23,24,24,24,24,24,24,24,24,67,68,69,70,71,72,73,74,75,76,77,78,79,80,81,82,83,1,1,1,1,1,1,1,1,12,13,14,15,24,24,24,24,24,24,24,24,24,24,37,38,39,1,1,1,16,17,18,24,24,24,67,68,69,70,71,72,73,74,75,76,77,78,79,80,81,82,83,1,7,8,9,10,11,24,24,45,46,47,48,49,50,51,52,53,54,55,1,1,1,16,17,18,24,24,24,24,24,24,40,41,42,43,44,1,1,1,1,1,1,1,1,1,16,17,18,24,24,25,26,27,28,29,30,1,1,1,1,1,1,1,1,12,13,14,15,24,24,37,38,39,1,19,20,21,22,23,24,24,24,24,67,68,69,70,71,72,73,74,75,76,77,78,79,80,81,82,83,1,1,1,1,16,17,18,24,24,24,67,68,69,70,71,72,73,74,75,76,77,78,79,80,81,82,83,1,1,1,1,1,2,3,4,5,6,24,24,24,45,46,47,48,49,50,51,52,53,54,55,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,7,8,9,10,11,24,24,67,68,69,70,71,72,73,74,75,76,77,78,79,80,81,82,83,1,1,1,1,7,8,9,10,11,24,24,37,38,39,1,1,1,1,1,1,16,17,18,24,24,24,24,24,24,24,24,24,24,24,24,31,32,33,34,35,36,1,7,8,9,10,11,24,24,24,24,24,24,56,57,58,59,60,61,62,63,64,65,66,1,1,1,1,1,12,13,14,15,24,24,24,24,24,56,57,58,59,60,61,62,63,64,65,66,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,19,20,21,22,23,24,24,24,24,24,24,24,24,24,24,24,24,24,24,25,26,27,28,29,30,1,1,1,1,16,17,18,24,24,24,45,46,47,48,49,50,51,52,53,54,55,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,7,8,9,10,11,24,24,24,24,40,41,42,43,44,1,1,7,8,9,10,11,24,45,46,47,48,49,50,51,52,53,54,55,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,2,3,4,5,6,24,24,67,68,69,70,71,72,73,74,75,76,77,78,79,80,81,82,83,1,12,13,14,15,24,24,67,68,69,70,71,72,73,74,75,76,77,78,79,80,81,82,83,1,1,1,2,3,4,5,6,24,24,24,40,41,42,43,44,1,1,1,1,19,20,21,22,23,24,24,37,38,39,1,1,12,13,14,15,24,24,24,25,26,27,28,29,30,1,1,1,1,1,1,1,2,3,4,5,6,24,67,68,69,70,71,72,73,74,75,76,77,78,79,80,81,82,83,1,1,1,1,12,13,14,15,24,24,24,24,24,24,24,24,24,67,68,69,70,71,72,73,74,75,76,77,78,79,80,81,82,83,1,1,1,1,1,1,1,12,13,14,15,24,24,24,24,24,24,24,24,31,32,33,34,35,36,1,1,1,1,19,20,21,22,23,24,24,24,24,24,24,24,24,31,32,33,34,35,36,1,1,1,1,1,1,1,1,1,19,20,21,22,23,24,24,24,24,24,24,67,68,69,70,71,72,73,74,75,76,77,78,79,80,81,82,83,1,1,1,1,1,1,7,8,9,10,11,24,24,24,67,68,69,70,71,72,73,74,75,76,77,78,79,80,81,82,83,1,16,17,18,24,24,24,24,24,24,24,24,24,24,24,24,56,57,58,59,60,61,62,63,64,65,66,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,16,17,18,24,24,24,24,24,24,24,45,46,47,48,49,50,51,52,53,54,55,1,1,1,1,1,1,1,1,1,7,8,9,10,11,24,24,45,46,47,48,49,50,51,52,53,54,55,1,2,3,4,5,6,24,24,24,24,24,24,24,24,24,24,24,24,24,24,24,24,24,25,26,27,28,29,30,1,19,20,21,22,23,24,24,67,68,69,70,71,72,73,74,75,76,77,78,79,80,81,82,83,1,1,1,1,1,12,13,14,15,24,24,24,56,57,58,59,60,61,62,63,64,65,66,1,1,1,1,1,1,1,1,1,19,20,21,22,23,24,24,24,24,24,24,24,24,24,24,24,37,38,39,1,1,1,1,1,2,3,4,5,6,24,24,24,24,40,41,42,43,44,1,1,2,3,4,5,6,24,40,41,42,43,44,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,2,3,4,5,6,24,45,46,47,48,49,50,51,52,53,54,55,1,1,1,1,1,1,1,1,1,1,1,1,12,13,14,15,24,24,24,24,24,24,24,24,24,40,41,42,43,44,1,1,2,3,4,5,6,24,24,24,56,57,58,59,60,61,62,63,64,65,66,1,1,1,1,1,1,1,1,1,1,1,1,1,1,2,3,4,5,6,24,24,24,25,26,27,28,29,30,1,12,13,14,15,24,24,24,45,46,47,48,49,50,51,52,53,54,55,1,1,7,8,9,10,11,24,24,45,46,47,48,49,50,51,52,53,54,55,1,1,7,8,9,10,11,24,24,24,24,24,24,24,25,26,27,28,29,30,1,1,1,1,1,1,1,1,7,8,9,10,11,24,45,46,47,48,49,50,51,52,53,54,55,1,16,17,18,24,24,24,24,24,24,24,24,24,25,26,27,28,29,30,1,1,1,12,13,14,15,24,67,68,69,70,71,72,73,74,75,76,77,78,79,80,81,82,83,1,1,7,8,9,10,11,24,24,24,45,46,47,48,49,50,51,52,53,54,55,1,1,1,1,1,1,1,12,13,14,15,24,40,41,42,43,44,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,19,20,21,22,23,24,24,24,24,24,24,24,24,24,25,26,27,28,29,30,1,16,17,18,24,24,24,24,56,57,58,59,60,61,62,63,64,65,66,1,1,1,1,1,1,1,1,1,16,17,18,24,24,25,26,27,28,29,30,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,12,13,14,15,24,24,24,24,24,24,24,24,24,24,24,24,24,67,68,69,70,71,72,73,74,75,76,77,78,79,80,81,82,83,1,1,12,13,14,15,24,24,24,24,24,24,24,24,24,24,24,25,26,27,28,29,30,1,1,1,1,1,1,1,1,1,1,19,20,21,22,23,24,24,24,24,24,24,24,24,24,24,24,40,41,42,43,44,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,2,3,4,5,6,24,37,38,39,1,1,1,1,1,1,19,20,21,22,23,24,56,57,58,59,60,61,62,63,64,65,66,1,19,20,21,22,23,24,24,24,24,24,24,24,24"
cs = cs.split(",")
for j, state_i in enumerate(cs):
    i = int(state_i)
    state = states[i-1]
    cs[j] = states_to_char[state]
#''.join(cs)

In [34]:
''.join(cs)

'       david        quinceadams    david     quinceadams      david          fitzwilliam                           jim fitzwilliam     barry            quinceadams    jim       fitzwilliam               barry  fitzwilliam                 anton    barber       david       chain     anton         ilsung          jim                ilsung            anton      grafvonunterhosen                       barry              ilsung      barry   grafvonunterhosen jim    fitzwilliam     david  chain     anton  quinceadams    barry   fitzwilliam     fred     grafvonunterhosen         fred   chain   fred grafvonunterhosen        fred grafvonunterhosen        anton               quinceadams     fred  barber    anton     fox  barry      chain           jim     grafvonunterhosen jim  fitzwilliam  barry      fitzwilliam      barry        ilsung                   barry   grafvonunterhosen       fred  grafvonunterhosen     fred      grafvonunterhosen               anton     chain                       ji

In [35]:
clean_sequence = ''.join(v)

In [36]:
elements = clean_sequence.split(" ")
names = []
for element in elements:
    if element != "":
        names.append(element)

In [37]:
for name in fnames + snames:
    n = clean_sequence.count(name)
    print("{}: {}".format(name, n))

david: 66
anton: 69
fred: 67
jim: 92
barry: 70
barber: 39
ilsung: 28
fox: 39
chain: 57
fitzwilliam: 73
quinceadams: 49
grafvonunterhosen: 78


In [38]:
occ = {}
for i, name in enumerate(names):
    if i % 2 == 1:
        first_name = names[i - 1]
        surname = names[i]
        if first_name not in occ:
            occ[first_name] = {}
        if surname not in occ[first_name]:
            occ[first_name][surname] = 0
        occ[first_name][surname] += 1
occ
    

{'david': {'quinceadams': 9,
  'fitzwilliam': 16,
  'chain': 11,
  'ilsung': 4,
  'grafvonunterhosen': 10,
  'fox': 8,
  'barber': 7},
 'jim': {'fitzwilliam': 20,
  'ilsung': 9,
  'grafvonunterhosen': 19,
  'fox': 8,
  'barber': 8,
  'chain': 16,
  'quinceadams': 12},
 'barry': {'quinceadams': 9,
  'fitzwilliam': 8,
  'grafvonunterhosen': 19,
  'chain': 10,
  'ilsung': 5,
  'fox': 10,
  'barber': 9},
 'anton': {'barber': 9,
  'ilsung': 5,
  'grafvonunterhosen': 11,
  'quinceadams': 11,
  'fox': 4,
  'fitzwilliam': 19,
  'chain': 10},
 'fred': {'grafvonunterhosen': 19,
  'chain': 10,
  'barber': 6,
  'quinceadams': 8,
  'fitzwilliam': 10,
  'fox': 9,
  'ilsung': 5}}